### Personal Information
- ID:     20127043
- Name:   Nguyen Thoai Dang Khoa
- Class:  20CLC08

### Import Library

In [466]:
import numpy as np
import matplotlib.pyplot as plt 
import math
from PIL import Image

### Open an image and convert to numpy arrays

In [467]:
def openImg():
  # Input 
  img = input('Enter name of an image: ')
  # Open img
  image = Image.open(img).convert("RGB")
  # Convert to numpy arrays (3d matrix)
  image = np.array(image)
  return image, img

### Change the brightness


In [468]:
def changeTheBrightness(image, bright = 50):
  # make a hard copy
  outputImage = image.copy()
  # shape decay
  rows = outputImage.shape[0]
  cols = outputImage.shape[1]
  channels = outputImage.shape[2]
  # adjust the brightness
  for y in range(rows):
    for x in range(cols):
        for c in range(channels):
          if (outputImage[y,x,c] + bright > 255):
            outputImage[y,x,c] = 255
          elif (outputImage[y,x,c] + bright < 0):
            outputImage[y,x,c] = 0
          else:
            outputImage[y,x,c] += bright 
  return outputImage

### Change the contrast

In [469]:
def changeTheContrast(image, contrast = 50): 
  # make a hard copy
  outputImage = image.copy()
  # shape decay
  rows = outputImage.shape[0]
  cols = outputImage.shape[1]
  channels = outputImage.shape[2]
  # adjust the contrast
  factor = (259 * (contrast + 255)) / (255 * (259 - contrast))
  actual_contrast = 0
  for y in range(rows):
    for x in range(cols):
        for c in range(channels):
          actual_contrast = factor*(outputImage[y,x,c] - 128) + 128
          if (actual_contrast > 255):
            outputImage[y,x,c] = 255
          elif (actual_contrast < 0):
            outputImage[y,x,c] = 0
          else:
            outputImage[y,x,c] = actual_contrast
  return outputImage
  

### Grayscale to RGB Conversion

In [470]:
def weightedGrayscaleMethod(colors):
    return (0.299*colors[0] + 0.587*colors[1] + 0.114*colors[2])

def avgGrayscaleMethod(colors):
    return (colors[0] + colors[1] + colors[2])/3

def convertToGrayscale(image, method = "weighted"):
  # make a hard copy
  outputImage = image.copy()
  # shape decay
  rows = outputImage.shape[0]
  cols = outputImage.shape[1]
  # convert to grayscale
  for i in range(rows): 
    for j in range(cols): 
        if (method == "weighted"):
            outputImage[i][j] = weightedGrayscaleMethod(outputImage[i][j])
        else:
            outputImage[i][j] = avgGrayscaleMethod(outputImage[i][j])
  return outputImage


### Flipping Image

In [471]:
def flippingImage(image, flipping_type = "vertically"): 
  # make a hard copy
  outputImage = image.copy()
  # flipping image
  if (flipping_type == "vertically"): outputImage = list(reversed(outputImage)) # flip vertically
  else:
    outputImage = [list(reversed(row)) for row in outputImage] # flip horizontally
  return np.array(outputImage)


### Blending Image

In [472]:
def blendingImage(image_1, image_2, alpha = 0.5):
  # initialize outputImage with 0
  outputImage = np.zeros(image_1.shape)
  # shape decay
  rows = image_1.shape[0]
  cols = image_1.shape[1]
  for i in range(rows):
    for j in range(cols):
      outputImage[i][j] = image_1[i][j] * (1.0 - alpha) + image_2[i][j] * alpha
  return outputImage

### Blur an image

In [473]:
# Box Blur kernel
box_kernel = [[1 / 9, 1 / 9, 1 / 9],
              [1 / 9, 1 / 9, 1 / 9],
              [1 / 9, 1 / 9, 1 / 9]]

# Gaussian kernel
gaussian_kernel = [[1/16, 1/8, 1/16],
                   [1/8, 1/4, 1/8],
                   [1/16, 1/8, 1/16]]

def blurringImage(image, kernel_type = "box_blur"): 
  # make a hard copy
  outputImage = image.copy()
  rows = outputImage.shape[0]
  cols = outputImage.shape[1]
  # create kernel
  if (kernel_type == "gauss_3x3"): kernel = gaussian_kernel
  else: kernel = box_kernel
  # Middle of the kernel
  mid_kernel = len(kernel) // 2
  kernel_size = len(kernel) 
  # Processing
  for i in range(mid_kernel, rows - mid_kernel):
      for j in range(mid_kernel, cols - mid_kernel):
          accumulator = np.array([0,0,0])
          for a in range(kernel_size):
              for b in range(kernel_size):
                  accumulator = accumulator + kernel[a][b]*image[i - mid_kernel + a][j - mid_kernel + b]
          outputImage[i][j] = accumulator
  return outputImage

### Crop image into Circle

In [474]:
def cropImageIntoCircle(image):
  # make a hard copy
  outputImage = image.copy()
  # shape decay
  rows = outputImage.shape[0]
  cols = outputImage.shape[1]
  channels = outputImage.shape[2]
  # radius
  a,b = rows / 2, cols / 2 
  # processing
  for i in range(rows):
    for j in range(cols):
      if (i - a) **2 + (j - b)**2 > (rows*cols)/4:
        for k in range(channels):
          outputImage[i,j,k] = 0
  return outputImage

### Cropp image into Ellipse

In [475]:
def cropImageIntoEllipse(image):
  # make a hard copy
  outputImage = image.copy()
  # shape decay
  rows = outputImage.shape[0]
  cols = outputImage.shape[1]
  channels = outputImage.shape[2]
  # ellipse center at (h,k)
  h,k = rows / 2, cols / 2
  # calculate factor
  a_factor = 1.25 # 316:256
  b_factor = 0.671875 # 172:256
  # calculate semi-major axis a and semi-minor axis b by multiplying factor
  a = h * a_factor
  b = k * b_factor
  # processing
  for i in range(rows):
    for j in range(cols):
      relativePosOfFirstEllipse = (math.cos(math.pi/4) ** 2 / a ** 2 + math.sin(math.pi/4) ** 2 / b ** 2)*(j - h)**2 + 2*math.cos(math.pi/4)*math.sin(math.pi/4)*(1/a**2 - 1/b**2)*(j - h)*(i - k) + (math.sin(math.pi/4) ** 2 / a ** 2 + math.cos(math.pi/4) ** 2 / b ** 2)*(i - k)**2
      relavetivePosOfsecondEllipse = (math.cos(-math.pi/4) ** 2 / a ** 2 + math.sin(-math.pi/4) ** 2 / b ** 2)*(j - h)**2 + 2*math.cos(-math.pi/4)*math.sin(-math.pi/4)*(1/a**2 - 1/b**2)*(j - h)*(i - k) + ( math.sin(-math.pi/4) ** 2 / a ** 2 + math.cos(-math.pi/4) ** 2 / b ** 2)*(i - k)**2
      if (relativePosOfFirstEllipse > 1 and relavetivePosOfsecondEllipse > 1): 
        for m in range(channels):
          outputImage[i,j,m] = 0
  return outputImage

### Displaying Images

In [476]:
# display 2 images side by side
def plot_two_images(img1, img2):
  _, ax = plt.subplots(1, 2, figsize=(20, 20))
  ax[0].imshow(img1)
  ax[0].axis('off')  # hide axis 
  ax[0].set_title('Original Image', fontsize = 16, fontweight = 'bold')
  ax[1].imshow(img2)
  ax[1].axis('off') # hide axis
  ax[1].set_title('Processed Image', fontsize = 16, fontweight = 'bold') 

# display 3 images side by side
def plot_three_images(img1, img2, img3):
  _, ax = plt.subplots(1, 3, figsize=(20, 20))
  ax[0].imshow(img1)
  ax[0].axis('off')  # hide axis 
  ax[0].set_title('First Image', fontsize = 16, fontweight = 'bold')
  ax[1].imshow(img2)
  ax[1].axis('off') # hide axis
  ax[1].set_title('Second Image', fontsize = 16, fontweight = 'bold')
  ax[2].imshow(img3)
  ax[2].axis('off') # hide axis
  ax[2].set_title('Processed Image', fontsize = 16, fontweight = 'bold')

### Export to file

In [477]:
def exportToFile(imageName, handleType, outputImage):
  imageName = imageName.split('.')
  outputImageName = f"{imageName[0]}_{handleType}.{imageName[1]}"
  Image.fromarray(outputImage.astype(np.uint8)).save(outputImageName)

### Post-processing (display image, export image)

In [478]:
def postProcessing(inputImage,outputImage, handleType, imageName):
  # display images
  plot_two_images(inputImage, outputImage)
  # Export to file
  exportToFile(imageName,handleType, outputImage)
  print("Image was successfully proccessed.")



def postProcessingForBending(inputImage1, inputImage2, outputImage, handleType, imageName1, imageName2):
   # display images
  plot_three_images(inputImage1,inputImage2,outputImage)
  # Export to file
  imageName1 = imageName1.split('.')
  imageName2 = imageName2.split('.')
  outputImageName = f"{imageName1[0]}_{imageName2[0]}_{handleType}.{imageName2[1]}"
  Image.fromarray(outputImage.astype(np.uint8)).save(outputImageName)
  print("Image was successfully proccessed.")



### Create a menu and handle option

In [479]:
menu_choices = {
    0: 'All Options',
    1: 'Adjust Brightness Of Image',
    2: 'Adjust Contrast Of Image',
    3: 'Flip An Image (Vertically / Horizontally)',
    4: 'Convert An RGB Image To A Grayscale',
    5: 'Blend Overlapping Images',
    6: 'Blur An Image',
    7: 'Crop An Image Into Circle',
    8: 'Crop An Image Into Ellipse',
    9: 'Exit'
}

def create_menu():
    for key in menu_choices.keys():
        print (f"{key}. {menu_choices[key]}")
    print("=================================================")


# handle option 1
def option1_handle(inputImage):
    print(f"Handle option 1. {menu_choices[1]}")
    bright = int(input('Enter the image brightness (from 255 to -255). If you choose 0, default 50 will be applied: '))
    if (bright != 0): outputImage = changeTheBrightness(inputImage, bright)
    else: outputImage = changeTheBrightness(inputImage)
    return outputImage
   


# handle option 2
def option2_handle(inputImage):
   print(f"Handle option 2. {menu_choices[2]}")
   contrast = int(input('Enter the image contrast (from 127 to -127). If you choose 0, default 50 will be applied: '))
   if (contrast != 0): outputImage = changeTheContrast(inputImage, contrast)
   else: outputImage = changeTheContrast(inputImage)
   return outputImage


# handle option 3
def option3_handle(inputImage):
   print(f"Handle option 3. {menu_choices[3]}")
   while True:
        flip_type = input("Enter type you want to flip an image (horizontally / vertically): ")
        if (flip_type == "horizontally" or flip_type == "vertically"): break
   outputImage = flippingImage(inputImage, flip_type)
   return outputImage, flip_type



# handle option 4
def option4_handle(inputImage):
   print(f"Handle option 4. {menu_choices[4]}")
   while True:
     method = input("Enter method you want to convert to a grayscale (average / weighted). If you choose 0, default mode will be applied: ")
     if (method == "average" or method == "weighted" or method == "0"): break
   if (method == "0"): outputImage = convertToGrayscale(inputImage)
   else: outputImage = convertToGrayscale(inputImage, method)
   return outputImage




# handle option 5
def option5_handle(inputImage1, imageName1):
    print(f"Handle option 5. {menu_choices[5]}")
    print(f"Size of image 1: {inputImage1.shape[1]} x {inputImage1.shape[0]}")
    print("The size image 2 should be the same as the size of image 1.")
    # Open img
    imageName2 = input('Enter name of an image 2: ')
    inputImage2 = Image.open(imageName2).convert("RGB")
    # if size of 2 images is not compatible with each other 
    # Convert to numpy arrays (3d matrix)
    inputImage2 = np.array(inputImage2)

    # convert 2 images to gray
    inputImage1 = convertToGrayscale(inputImage1)
    inputImage2 = convertToGrayscale(inputImage2)

    # blending 2 images
    outputImage = blendingImage(inputImage1, inputImage2).astype(np.uint8)
    return inputImage1, inputImage2, outputImage, imageName1, imageName2

# handle option 6
def option6_handle(inputImage):
   print(f"Handle option 6. {menu_choices[6]}")
   while True:
     kernel = input("Enter type of kernel used for blurring (box_blur / gauss_3x3). If you choose 0, default mode will be applied: ")
     if (kernel == "box_blur" or kernel == "gauss_3x3" or kernel == "0"): break
   if (kernel == "0"): outputImage = blurringImage(inputImage)
   else: outputImage = blurringImage(inputImage, kernel)
   return outputImage
   
# handle option 7
def option7_handle(inputImage):
   print(f"Handle option 7. {menu_choices[7]}")
   outputImage = cropImageIntoCircle(inputImage)
   return outputImage

# handle option 8
def option8_handle(inputImage):
   print(f"Handle option 8. {menu_choices[8]}")
   outputImage = cropImageIntoEllipse(inputImage)
   return outputImage


# handle all options
def option_handle_all(inputImage, imageName):
  # handle option 1
  outputImage = option1_handle(inputImage)
  exportToFile(imageName,"bright", outputImage)
  print('Option 1 finished!')

  # handle option 2
  outputImage = option2_handle(inputImage)
  exportToFile(imageName,"contrast", outputImage)
  print('Option 2 finished!')

  # handle option 3
  outputImage, flip_type = option3_handle(inputImage)
  exportToFile(imageName,f"flip_{flip_type}", outputImage)
  print('Option 3 finished!')
  
  # handle option 4
  outputImage = option4_handle(inputImage)
  exportToFile(imageName,"gray", outputImage)
  print('Option 4 finished!')

  # handle option 5
  inputImage1, inputImage2, outputImage, imageName1, imageName2 = option5_handle(outputImage, imageName)
  imageName2 = imageName2.split('.')
  imageName1 = imageName1.split('.')
  outputImageName = f"{imageName1[0]}_{imageName2[0]}.{imageName2[1]}"
  exportToFile(outputImageName,"blending", outputImage)
  print('Option 5 finished!')

  # handle option 6
  outputImage = option6_handle(inputImage)
  exportToFile(imageName,"blur", outputImage)
  print('Option 6 finished!')

  # handle option 7
  outputImage = option7_handle(inputImage)
  exportToFile(imageName,"circle", outputImage)
  print('Option 7 finished!')

  # handle option 8
  outputImage = option8_handle(inputImage)
  exportToFile(imageName,"ellipse", outputImage)
  print('Option 8 finished!')

### Main Function

In [480]:
import sys
def main():
   #while(True):
    create_menu()
    option = ''
    try:
        option = int(input('Enter your choice: '))
    except:
        print('Wrong input. Please enter a number between 0 and 7.')
    #Check what option was inputted and execute accordingly
    inputImage, imageName = openImg()
    if option == 0:
        option_handle_all(inputImage, imageName)
    elif option == 1:
        outputImage = option1_handle(inputImage) # handle option 1
        postProcessing(inputImage,outputImage,"bright",imageName) # post-processing
    elif option == 2:
        outputImage = option2_handle(inputImage) # handle option 2
        postProcessing(inputImage,outputImage,"contrast",imageName) # post-processing
    elif option == 3:
        outputImage, flip_type = option3_handle(inputImage) # handle option 3
        postProcessing(inputImage,outputImage,f"flip_{flip_type}",imageName) # post-processing
    elif option == 4:
        outputImage = option4_handle(inputImage) # handle option 4
        postProcessing(inputImage,outputImage,"gray",imageName) # post-processing
    elif option == 5:
        inputImage1, inputImage2, outputImage, imageName1, imageName2 = option5_handle(inputImage, imageName) # handle option 5
        postProcessingForBending(inputImage1, inputImage2, outputImage, "blending", imageName1, imageName2) # post-processing
    elif option == 6:
        outputImage = option6_handle(inputImage) # handle option 6
        postProcessing(inputImage,outputImage,"blur",imageName) # post-processing
    elif option == 7:
        outputImage = option7_handle(inputImage) # handle option 7
        postProcessing(inputImage, outputImage, "circle", imageName)  # post-processing
    elif option == 8:
        outputImage = option8_handle(inputImage) # handle option 8
        postProcessing(inputImage, outputImage, "ellipse", imageName) # post-processing
    elif option == 9:
        print('Thank you for your time!')
        sys.exit()
    else:
        print('Invalid option. Please enter a number between 0 and 7.')

# calling main function
main()

0. All Options
1. Adjust Brightness Of Image
2. Adjust Contrast Of Image
3. Flip An Image (Vertically / Horizontally)
4. Convert An RGB Image To A Grayscale
5. Blend Overlapping Images
6. Blur An Image
7. Crop An Image Into Circle
8. Crop An Image Into Ellipse
9. Exit
Enter your choice: 0
Enter name of an image: lena.jpg
Handle option 1. Adjust Brightness Of Image
Enter the image brightness (from 255 to -255). If you choose 0, default 50 will be applied: 0
Option 1 finished!
Handle option 2. Adjust Contrast Of Image
Enter the image contrast (from 127 to -127). If you choose 0, default 50 will be applied: 0
Option 2 finished!
Handle option 3. Flip An Image (Vertically / Horizontally)
Enter type you want to flip an image (horizontally / vertically): vertically
Option 3 finished!
Handle option 4. Convert An RGB Image To A Grayscale
Enter method you want to convert to a grayscale (average / weighted). If you choose 0, default mode will be applied: 0
Option 4 finished!
Handle option 5. Blen